In [106]:
import nest
import datetime
import numpy as np
import os
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation

In [116]:
client_id = '1f502532-26b1-4567-8c3a-c78ab184039a'
client_secret = 'RBsVN6rSgNOTL0e7gl1qUx40v'
access_token_cache_file = 'nest.json'
model_file = 'model.h5'
target_file = 'target.npy'

In [100]:
napi = nest.Nest(client_id=client_id, client_secret=client_secret, access_token_cache_file=access_token_cache_file)

In [101]:
if napi.authorization_required:
    print("Please supply PIN")
    print(napi.authorize_url)

In [103]:
pin = False

In [104]:
if pin:
    napi.request_token('N95GDHUA')

In [105]:
lr = napi.structures[0].thermostats[0]

In [107]:
if os.path.isfile(model_file):
    print("Loading model")
    model = load_model(model_file)
else:
    print("Generating model")
    model = Sequential()
    model.add(Dense(64, input_dim=6))
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

In [117]:
if os.path.isfile(target_file):
    print("Loading saved target file")
    data = np.load(target_file)
else:
    print("Generating new target file")
    data = np.array([datetime.datetime.now(), lr.target])
    np.save(target_file, data)

In [120]:
def input_fn(lr, last_change):
    now = datetime.datetime.now()
    diff = now - last_change
    return np.array([[now.hour/24.0, now.minute/60.0, 
                     diff.seconds/3600/24.0, (diff.seconds/60)%60/60.0,
                     lr.temperature/100.0, lr.humidity/100.0]]), np.array([[lr.target/100.0]])

In [121]:
x, y = input_fn(lr, data[0])

In [122]:
if data[1] != lr.target:
    print("Target was updated {} {}".format(data[1], lr.target))
    # Setting changed, learn
    data = np.array([datetime.datetime.now(), lr.target])
    np.save(target_file, data)

In [123]:
x, y, data

(array([[ 0.66666667,  0.7       ,  0.        ,  0.08333333,  0.24      ,
          0.5       ]]),
 array([[ 0.27]]),
 array([datetime.datetime(2017, 9, 25, 16, 37, 11, 332383), 27.0], dtype=object))

In [125]:
model.fit(x, y, nb_epoch=10, batch_size=1)

Epoch 1/10
1/1 [==============================] - 0s - loss: 0.1802
Epoch 2/10
1/1 [==============================] - 0s - loss: 0.1496
Epoch 3/10
1/1 [==============================] - 0s - loss: 0.1225
Epoch 4/10
1/1 [==============================] - 0s - loss: 0.0986
Epoch 5/10
1/1 [==============================] - 0s - loss: 0.0777
Epoch 6/10
1/1 [==============================] - 0s - loss: 0.0597
Epoch 7/10
1/1 [==============================] - 0s - loss: 0.0444
Epoch 8/10
1/1 [==============================] - 0s - loss: 0.0320
Epoch 9/10
1/1 [==============================] - 0s - loss: 0.0220
Epoch 10/10
1/1 [==============================] - 0s - loss: 0.0141


In [127]:
new_target = int(model.predict(x) * 100.0)

In [128]:
new_target

17

In [131]:
if new_target < lr.temperature:
    print("Updating temperature")
    lr.target = new_target

Updating temperature


In [133]:
model.save(model_file)